<a href="https://colab.research.google.com/github/revanthmadasu/machine-learning/blob/nallam-project/nallam-project1/Project-1-starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 Starter

**This is draft - version 0 - changes are possible and will be anounced.**

Project 1 is to allow students to practice Data Science concepts learned so far.

The project will include following tasks:
- Load dataset. Don't use "index" column for training.
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Validation/Test
- Add engineered features
- Train and tune ML model
- Provide final metrics using Test dataset

### Types of models to train

Your final submission should include single model. 
The model set you should try to come up with best model:
1. Sklearn Logistic Regression - try all combinations of regularization
2. H2O-3 GLM - try different combinations of regularization



### Feature engineering

You should train/fit categorical features scalers and encoders on Train only. Use `transform` or equivalent function on Validation/Test datasets.

It is important to understand all the steps before model training, so that you can reliably replicate and test them to produce scoring function.


You should generate various new features. Examples of such features can be seen in the Module-3 lecture on GLMs.
Your final model should have at least **10** new engineered features. On-hot-encoding, label encoding, and target encoding is not included in the **10** features.
You can try, but target encoding is not expected to produce improvement for Linear models.

Ideas for Feature engineering for various types of variables:
1. https://docs.h2o.ai/driverless-ai/1-10-lts/docs/userguide/transformations.html
2. GLM lecture and hands-on (Module-3)


**Note**: 
- You don't have to perform feature engineering using H2O-3 even if you decided to use H2O-3 GLM for model training.
- It is OK to perfor feature engineering using any technique, as long as you can replicate it correctly in the Scoring function.


### Threshold calculation

You will need to calculate optimal threshold for class assignment using F1 metric:
- If using sklearn, use F1 `macro`: `f1_score(y_true, y_pred, average='macro')` 
- If using H2O-3, use F1

You will need to find optimal probability threshold for class assignment, the threshold that maximizes above F1.



### Scoring function

The Project-1 will be graded based on the completeness and performance of your final model against the hold-out dataset.
The hold-out dataset will not be known to the students. As part of your deliverables, you will need to submit a scoring function. The scoring function will perform the following:
- Accept dataset in the same format as provided with the project, minus "MIS_Status" column
- Load trained model and any encoders/scalers that are needed to transform data
- Transform dataset into format that can be scored with the trained model
- Score the dataset and return the results, for each record
    - Record ID
    - Record label as determined by final model (0 or 1)
    - If your model returns probabilities, you need to assign the label based on maximum F1 threshold
    
Scoring function header:
```
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    l = data.shape[0]
    return l*[0]
```

Look for full example of scoring function at the bottom of the notebook. **Don't copy as is - this is just an example**



### Deliverables in a single zip file in the following structure:
- `notebook` (folder)
    - Jupyter notebook with complete code to manipulate data, train and tune final model. `ipynb` format
    - Jupyter notebook in `html` format
- `artifacts` (folder)
    - Model and any potential encoders in the "pkl" format or native H2O-3 format (for H2O-3 model)
    - Scoring function that will load the final model and encoders. Separate from above notebook or `.py` file



Your notebook should include explanations about your code and be designed to be easily followed and results replicated. Once you are done with the final version, you will need to test it by running all cells from top to bottom after restarting Kernel. It can be done by running `Kernel -> Restart & Run All`


**Important**: To speed up progress, first produce working code using a small subset of the dataset.

## Dataset description

The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

**Don't use original dataset, use only dataset provided with project requirements in eLearning**

## Preparation

Use dataset provided in the eLearning

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

### Load data

In [4]:
data = pd.read_csv('./data/SBA_loans_project_1.zip')

In [ ]:
print("Data shape:", data.shape)

Data shape: (809247, 21)


**Review dataset**

In [ ]:
desc_df = describe_more(data)
desc_df

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,index,809247,int64,1,6,"{0: 1, 539491: 1, 539493: 1, 539494: 1, 539495..."
1,City,31320,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
2,State,51,object,2,3,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
3,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
4,Bank,5716,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
5,BankState,55,object,2,3,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
6,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
7,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
8,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
9,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 932}"


## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

Any additional clean-up as you find fit.

In [5]:
# remove index
processed_data = data.loc[:,data.columns!='index']
def dollar_string_to_float(dollar_str):
  dollar_str = dollar_str.replace(',', '')
  dollar_float = float(dollar_str[1:])
  return dollar_float
currency_cols = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
for colname in currency_cols:
  processed_data[colname] = processed_data[colname].apply(dollar_string_to_float)
processed_data.loc[0]

City                                  GLEN BURNIE
State                                          MD
Zip                                         21060
Bank                 BUSINESS FINANCE GROUP, INC.
BankState                                      VA
NAICS                                      811111
Term                                          240
NoEmp                                           7
NewExist                                      1.0
CreateJob                                       6
RetainedJob                                     7
FranchiseCode                                   1
UrbanRural                                      1
RevLineCr                                       0
LowDoc                                          N
DisbursementGross                        743000.0
BalanceGross                                  0.0
GrAppv                                   743000.0
SBA_Appv                                 743000.0
MIS_Status                                  P I F


In [6]:
processed_data['MIS_Status'] = processed_data['MIS_Status'].replace('P I F', 0)
processed_data['MIS_Status'] = processed_data['MIS_Status'].replace('CHGOFF', 1)
processed_data = processed_data.dropna(subset='MIS_Status')
processed_data['MIS_Status'].unique()

array([0., 1.])

In [11]:
for column in processed_data.columns:
  print(column)
  print(processed_data[column].isna().sum())

City
0
State
0
Zip
0
Bank
0
BankState
0
NAICS
0
Term
0
NoEmp
0
NewExist
0
CreateJob
0
RetainedJob
0
FranchiseCode
0
UrbanRural
0
RevLineCr
0
LowDoc
0
DisbursementGross
0
BalanceGross
0
GrAppv
0
SBA_Appv
0
MIS_Status
0


In [ ]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809247 entries, 0 to 809246
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   City               809222 non-null  object 
 1   State              809235 non-null  object 
 2   Zip                809247 non-null  int64  
 3   Bank               807842 non-null  object 
 4   BankState          807836 non-null  object 
 5   NAICS              809247 non-null  int64  
 6   Term               809247 non-null  int64  
 7   NoEmp              809247 non-null  int64  
 8   NewExist           809119 non-null  float64
 9   CreateJob          809247 non-null  int64  
 10  RetainedJob        809247 non-null  int64  
 11  FranchiseCode      809247 non-null  int64  
 12  UrbanRural         809247 non-null  int64  
 13  RevLineCr          805153 non-null  object 
 14  LowDoc             806928 non-null  object 
 15  DisbursementGross  809247 non-null  float64
 16  Ba

In [8]:
# handling null data
row_null_counts = processed_data.isna().any(axis=1).sum()
print("\nNumber of rows with null values:", row_null_counts)
def check_null_counts():
  col_null_counts = dict()
  for column in processed_data.columns:
    col_null_count = processed_data[column].isna().sum()
    if col_null_count:
      col_null_counts[column] = col_null_count
  return col_null_counts
print(f'before handling: {check_null_counts()}')
categoral_null_cols = ['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc']
for null_cat_col in categoral_null_cols:
  processed_data[null_cat_col] = processed_data[null_cat_col].fillna('unknown').astype('category')
processed_data['NewExist'] = processed_data['NewExist'].fillna(0)
print(f'after handling handling: {check_null_counts()}')


Number of rows with null values: 7915
before handling: {'City': 25, 'State': 11, 'Bank': 1359, 'BankState': 1365, 'NewExist': 127, 'RevLineCr': 4086, 'LowDoc': 2315}
after handling handling: {}


In [9]:
processed_data['MIS_Status'].value_counts()

0.0    665576
1.0    141849
Name: MIS_Status, dtype: int64

In [10]:
# print(f'before: {processed_data['RevLineCr'].value_counts()}')
for replace_cat in ['1', 'R', '`', '2', 'C', 'Q', '-', '.', '7', '5', 'C', 'Q','.','7','5','3','A' ]:
  processed_data['RevLineCr'] = processed_data['RevLineCr'].replace(replace_cat, 'unknown')
processed_data['RevLineCr'].value_counts()

N          377487
0          231812
Y          180329
T           13653
unknown      4144
Name: RevLineCr, dtype: int64

In [ ]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809247 entries, 0 to 809246
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   City               809247 non-null  category
 1   State              809247 non-null  category
 2   Zip                809247 non-null  int64   
 3   Bank               809247 non-null  category
 4   BankState          809247 non-null  category
 5   NAICS              809247 non-null  int64   
 6   Term               809247 non-null  int64   
 7   NoEmp              809247 non-null  int64   
 8   NewExist           809247 non-null  float64 
 9   CreateJob          809247 non-null  int64   
 10  RetainedJob        809247 non-null  int64   
 11  FranchiseCode      809247 non-null  int64   
 12  UrbanRural         809247 non-null  int64   
 13  RevLineCr          809247 non-null  category
 14  LowDoc             809247 non-null  category
 15  DisbursementGross  809247 non-null

## Categorical and numerical variables encoding

Encode categorical variables using either one of the techniques below. Don't use LabelEncoder.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- WOE encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_woe


WOE encoder can be used with numerical variables too. 


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [12]:
s_processed_data = processed_data.sample(frac=0.01, random_state=42)
s_processed_data
print(s_processed_data.shape)

[['PROVIDENCE' 'RI' 2906 ... 0.0 40000.0 20000.0]
 ['FRESNO' 'CA' 93701 ... 0.0 100000.0 85000.0]
 ['PINELLAS PARK' 'FL' 33781 ... 0.0 35000.0 17500.0]
 ...
 ['WILLISTON' 'ND' 58801 ... 0.0 70000.0 59500.0]
 ['ROSWELL' 'GA' 30076 ... 0.0 120000.0 108000.0]
 ['LAFAYETTE' 'CA' 94549 ... 0.0 382400.0 286800.0]]
[0. 0. 1. ... 0. 0. 0.]
(8074, 20)


In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# X_transformed = np.array(ct.fit_transform(X))
encoded_data = pd.get_dummies(s_processed_data, columns=['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc', 'Zip', 'MIS_Status', 'NAICS', 'NewExist', 'FranchiseCode', 'UrbanRural'])
encoded_data.iloc[0]

Term                      84.0
NoEmp                      2.0
CreateJob                  1.0
RetainedJob                0.0
DisbursementGross      65341.0
                        ...   
FranchiseCode_90979        0.0
FranchiseCode_91297        0.0
UrbanRural_0               0.0
UrbanRural_1               1.0
UrbanRural_2               0.0
Name: 358619, Length: 43638, dtype: float64

In [50]:
encoded_data.dtypes

Term                     int64
NoEmp                    int64
CreateJob                int64
RetainedJob              int64
DisbursementGross      float64
                        ...   
FranchiseCode_90979      uint8
FranchiseCode_91297      uint8
UrbanRural_0             uint8
UrbanRural_1             uint8
UrbanRural_2             uint8
Length: 43638, dtype: object

In [51]:
from sklearn.model_selection import train_test_split
X = encoded_data.iloc[:, :-1].values
y = encoded_data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X)
print(y)

[[ 84.   2.   1. ...   0.   0.   1.]
 [120.   1.   0. ...   0.   0.   1.]
 [ 55.   4.   0. ...   0.   0.   1.]
 ...
 [ 72.  14.   0. ...   0.   1.   0.]
 [300.   6.   0. ...   0.   1.   0.]
 [300.   4.   0. ...   0.   1.   0.]]
[0 0 0 ... 0 0 0]


# Model Training

See Project summary for types of models

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8804953560371517


## Model Tuning

Choose one model from the above list. You should provide reasoning on why you have picked the model over others. Perform tuning for the selected model:
- Hyper-parameter tuning. Your hyper-parameter search space should have at least 50 combinations.
- To avoid overfitting and provide you with reasonable estimate of model performance on hold-out dataset, you will need to split your dataset as following:
    - Train, will be used to train model
    - Validation, will be used to validate model each round of training
    - Testing, will be used to provide final performance metrics, used only once on the final model
- Feature engineering. See project description

**Selelct final model that produces best performance on the Test dataset.**
- For the best model, calculate probability threshold to maximize F1. 

## Save all artifacts

Save all artifacts needed for scoring function:
- Trained model
- Encoders
- Any other arficats you will need for scoring

**You should stop your notebook here. Scoring function should be in a separate file/notebook.**

## Stop Here. Create new file/notebook

## ==============================================

## Model Scoring

Write function that will load artifacts from above, transform and score on a new dataset.
Your function should return Python list of labels. For example: [0,1,0,1,1,0,0]


In [ ]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return pandas DF with following columns:
            - index
            - label
            - probability_0
            - probability_1
    """
    pass

### Example of Scoring function

Don't copy the code as is. It is provided as an example only. 
- Function `train_model` - you need to focus on model and artifacts saving:
    ```
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    ```
- Function `project_1_scoring` - you should have similar function with name `project_1_scoring`. The function will:
    - Get Pandas dataframe as parameter
    - Will load model and all needed encoders
    - Will perform needed manipulations on the input Pandas DF - in the exact same format as input file for the project, minus MIS_Status feature
    - Return Pandas DataFrame
        - record index
        - predicted class for threshold maximizing F1
        - probability for class 0 (PIF)
        - probability for class 1 (CHGOFF)


In [ ]:
"""
Don't copy of use the cell code in any way!!!
The code is provided as an example of generating artifacts for scoring function
Your scoring function code should not have model training part!!!!
"""
import pandas as pd
import numpy as np
def train_model(data):
    """
    Train sample model and save artifacts
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    from sklearn.impute import SimpleImputer
    
    target_col = "Survived"
    cols_to_drop = ['Name', 'Ticket', 'Cabin','SibSp', 'Parch', 'Sex','Embarked','PassengerId','Survived']
    y = data[target_col]
    X = data.drop(columns=[target_col])
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    imp_age_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_age_mean.fit(X[['Age']])
    X['Age'] = imp_age_mean.transform(X[['Age']])


    ohe_orig_columns = ["Embarked","Sex"]
    cat_encoders = {}
    for col in ohe_orig_columns:
        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        enc.fit(X[[col]])
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X= pd.concat([X, result_train], axis=1)
        cat_encoders[col] = [deepcopy(enc),"ohe"]
        
    clf = LogisticRegression(max_iter=1000, random_state=0)
    
    columns_to_train = [x for x in X.columns if x not in cols_to_drop]
    clf.fit(X[columns_to_train], y)
    
    # Todo: Add code to calculate optimal threshold. Replace 0.5 !!!!!
    threshold = 0.5
    # End Todo
    
    artifacts_dict = {
        "model": clf,
        "cat_encoders": cat_encoders,
        "imp_age_mean": imp_age_mean,
        "ohe_columns": ohe_orig_columns,
        "columns_to_train":columns_to_train,
        "threshold": threshold
    }
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "wb")
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    
    artifacts_dict_file.close()    
    return clf

In [ ]:
df_train = pd.read_csv('titanic.csv')
train_model(df_train)

LogisticRegression(max_iter=1000, random_state=0)

### Example scoring function

This is example only. Don't copy code as is!!!   
You must place scoring function in a separate Python file or Jupyter notebook.   

**Don't place function in the same notebook as rest of the code**

In [ ]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    
    X = data.copy()
    
    '''Load Artifacts'''
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    
    clf = artifacts_dict["model"]
    cat_encoders = artifacts_dict["cat_encoders"]
    imp_age_mean = artifacts_dict["imp_age_mean"]
    ohe_columns = artifacts_dict["ohe_columns"]
    columns_to_score = artifacts_dict["columns_to_train"]
    threshold = artifacts_dict["threshold"]
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    X['Age'] = imp_age_mean.transform(X[['Age']])
    
    '''Encode categorical columns'''
    for col in ohe_columns:
        enc = cat_encoders[col][0]
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X = pd.concat([X, result_train], axis=1)
        
    y_pred_proba = clf.predict_proba(X[columns_to_score])
    y_pred = (y_pred_proba[:,0] < threshold).astype(np.int16)
    d = {"index":data["PassengerId"],
         "label":y_pred,
         "probability_0":y_pred_proba[:,0],
         "probability_1":y_pred_proba[:,1]}
    
    return pd.DataFrame(d)

In [ ]:
project_1_scoring(df_train).head()

,index,label,probability_0,probability_1
0,1,0,0.901298,0.098702
1,2,1,0.071879,0.928121
2,3,1,0.367665,0.632335
3,4,1,0.098564,0.901436
4,5,0,0.923460,0.076540
